In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statistics
src_path = r"D:/Codes/GIthub_repo/Energy_grid/data/UCSD_raw_data/PV_ALL.csv"

In [ ]:
data=pd.read_csv(src_path)
data=data.drop_duplicates()
data=data.sort_values(by="DateTime")
data["DateTime"]=pd.to_datetime(data["DateTime"])
data=data.set_index("DateTime")
pred=data.sort_values(by="DateTime")

In [ ]:
data

In [ ]:
#2004-12-1是时间序列开始的时间，并以月为间隔
rng = pd.date_range(start=data.index.min(),end=data.index.max(), freq = '15min')
pd_rng=pd.DataFrame(rng)
s = pd.Series(rng)

In [ ]:
type(pd_rng)

In [ ]:
pd_rng=pd_rng.rename(columns={0:'DateTime'}).copy()

In [ ]:
pd_rng

In [ ]:
data=pd.merge(data,pd_rng,on='DateTime',how='right')

In [ ]:
data

In [ ]:
data['pred'] = data.apply(lambda x: [], axis=1)

In [ ]:
data_n=data.loc[:959]

In [ ]:
len(data_n)

In [ ]:
data_n['DateTime']=pd.to_datetime(data_n['DateTime'])
data_n=data_n.set_index('DateTime')

In [ ]:
data_n

In [ ]:
K=10
days_ahead=3
data_n['historical_ave']=None
for i in data_n.index:
    history=[]
    for k in range(days_ahead):
        index_ahead=i-timedelta(days=int(k+1))
        if index_ahead in data_n.index:
            history.append(data_n.at[index_ahead,'RealPower'])
        #else:
            #history.append(data_n.at[i,'RealPower'])
        
    data_n.at[i,'historical_ave']=np.mean(history)


    
    

In [ ]:
data_n=data_n.reset_index()

In [ ]:
data_n.info()

In [ ]:
type(data_n.index[0])

In [ ]:
alpha=0.1
last_weights = np.exp(- alpha*(np.arange(K)+1)) if alpha is not None else 0

In [ ]:
last_weights

In [ ]:
data_n

In [ ]:
alpha=0.1
for n in range(len(data_n)-K-1):
    # "realpower" 1st column
    last=data_n.iat[n,1]
    last_weights = np.exp(- alpha*(np.arange(K)+1)) if alpha is not None else 0
    for i in range(K):
        #"historical_ave" 3rd column, 'Pred' 2nd column
        pred_value=last_weights[i]*last+(1-last_weights[i])*data_n.iat[n+i,3]
        data_n.iat[n+i,2].append(pred_value)

In [ ]:
data_n.head(-50)

In [ ]:
data_n['pred']=data_n['pred'].map(lambda x: np.nanmean(x))

In [ ]:
data_n['pred_error']=data_n[["RealPower","pred"]].apply(lambda x: abs(x["RealPower"]-x["pred"]),axis=1)

In [ ]:
data_n.head(-50)

In [215]:
def get_prediction_Simple(src_path,start_t,end_t,days_ahead,save_path,K,alpha):
    
    data=pd.read_csv(src_path)
    data=data.drop_duplicates()
    data=data.sort_values(by="DateTime")
    data["DateTime"]=pd.to_datetime(data["DateTime"])
    data=data.set_index("DateTime")
    pred=data[start_t:end_t]
    pred=pred.sort_values(by="DateTime")

    #helper dataframe
    rng = pd.date_range(start=pred.index.min(),end=pred.index.max(), freq = '15min')
    pd_rng=pd.DataFrame(rng)
    pd_rng=pd_rng.rename(columns={0:'DateTime'}).copy()

    data_n=pd.merge(pred,pd_rng,on='DateTime',how='right')
    data_n['RealPower_pred'] = data_n.apply(lambda x: [], axis=1)
    data_n['DateTime']=pd.to_datetime(data_n['DateTime'])
    data_n=data_n.set_index('DateTime')
    data_n['RealPower']=data_n['RealPower'].astype(float)
    for i in data_n.index:
        if not isinstance(data_n.at[i,'RealPower'],float):
            data_n.at[i,'RealPower']=None
    
    data_n['historical_ave']=None
    for i in data_n.index:
        history=[]
        for k in range(days_ahead):
            index_ahead=i-timedelta(days=int(k+1))
            if index_ahead in data_n.index:
                history.append(data_n.at[index_ahead,'RealPower'])
            #else:
                #history.append(Null)                 
        ave=0
        count=0
        for m in history:
            if isinstance(m,(str,float,int)):
                ave=ave+float(m)
                count=count+1
        if count>0:
            ave=ave/count
            data_n.at[i,'historical_ave']=ave
    data_n=data_n.reset_index()
    
    last_weights = np.exp(- alpha*(np.arange(K)+1)) if alpha is not None else 0

    if 'ReactivePower' in data_n.columns:
        data_n=data_n.drop(columns=['ReactivePower'])
    print(data_n.head(50))
    
    for n in range(len(data_n)-K-1):
        # "realpower" 1st column
        last=data_n.iat[n,1]
        if not isinstance(last,float):
            last=None
        #last_weights = np.exp(- alpha*(np.arange(K)+1)) if alpha is not None else 0
        for i in range(K):
            #"historical_ave" 3rd column, 'Pred' 2nd column
            share=last_weights[i].copy()
            if not isinstance(share,float):
                print(last_weights)
                print(type(last_weights))
                raise Exception('invalid type of last_weights')
            if not isinstance(last,(float,None)):
                print(last)
                print(type(last))
                raise Exception('invalid type of last')  
            if  data_n.iat[n+i,3] != None:       
                if last != None:
                    pred_value=share*last+(1-share)*data_n.iat[n+i,3]
                else:
                    pred_value=data_n.iat[n+i,3]
                data_n.iat[n+i,2].append(pred_value)
    
    data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))
    data_n['pred_error']=data_n[["RealPower","RealPower_pred"]].\
        apply(lambda x: abs(x["RealPower"]-x["RealPower_pred"]),axis=1)
    
    ''' OLDer version of code:
    
    #pred=pred.set_index("DateTime")
    #pred.index = pd.to_datetime(pred.index)
    pred["RealPower_pred"]=None
    for i in pred.index:
        history=[]
        for k in range(days_ahead):
            index_ahead=i-timedelta(days=k+1)
            if index_ahead in pred.index:
                history.append(pred.loc[index_ahead,'RealPower'])
            else:
                history.append(pred.loc[i,'RealPower'])
        
        pred.loc[i,'RealPower_pred']=np.mean(history)
    '''
    data_n.to_csv(save_path,index=True)
    return data_n

In [216]:

blds=[
 #'CenterHall',# deleted
 #'EastCampus',# deleted
 'GalbraithHall',
 #'GeiselLibrary',# deleted
 #'Gilman',# missing of test sets
 'Hopkins',
 'Mandeville',
 'MusicBuilding',
 #'OttersonHall',# missing of test sets
 #'PepperCanyon',# missing of test sets
 #'PoliceDepartment', deleted
 'RadyHall',
 'RobinsonHall',
 'SocialScience',
 #'StudentServices' deleted
]
for i in blds:
    get_prediction_Simple(
        src_path=r'D:\Codes\GIthub_repo\Energy_grid\data\UCSD_raw_data\BLD_'+i+'.csv',
        days_ahead=7,
        K=96,
        alpha=0.1,
        start_t='01-Jan-2019',
        end_t='31-Dec-2019',
        save_path = r'D:\Codes\GIthub_repo\Energy_grid\scripts\XGboost_Optuna\Simple_stand_alone\BLD_'+i+'Simple_pred.csv',
    )

              DateTime  RealPower RealPower_pred historical_ave
0  2019-01-01 00:00:00    121.242             []           None
1  2019-01-01 00:15:00    115.773             []           None
2  2019-01-01 00:30:00    113.093             []           None
3  2019-01-01 00:45:00    111.401             []           None
4  2019-01-01 01:00:00    116.448             []           None
5  2019-01-01 01:15:00    120.667             []           None
6  2019-01-01 01:30:00    121.255             []           None
7  2019-01-01 01:45:00    121.994             []           None
8  2019-01-01 02:00:00    112.354             []           None
9  2019-01-01 02:15:00    112.414             []           None
10 2019-01-01 02:30:00    111.501             []           None
11 2019-01-01 02:45:00    111.881             []           None
12 2019-01-01 03:00:00    112.551             []           None
13 2019-01-01 03:15:00    117.543             []           None
14 2019-01-01 03:30:00    121.630       

C:\Users\Lunlo\AppData\Local\Temp\ipykernel_6060\392398549.py:75: RuntimeWarning: Mean of empty slice
  data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))


              DateTime  RealPower RealPower_pred historical_ave
0  2019-01-01 00:00:00     44.292             []           None
1  2019-01-01 00:15:00     44.472             []           None
2  2019-01-01 00:30:00     44.209             []           None
3  2019-01-01 00:45:00     44.409             []           None
4  2019-01-01 01:00:00     45.181             []           None
5  2019-01-01 01:15:00     44.511             []           None
6  2019-01-01 01:30:00     44.312             []           None
7  2019-01-01 01:45:00     44.533             []           None
8  2019-01-01 02:00:00     44.801             []           None
9  2019-01-01 02:15:00     44.480             []           None
10 2019-01-01 02:30:00     44.412             []           None
11 2019-01-01 02:45:00     44.475             []           None
12 2019-01-01 03:00:00     44.848             []           None
13 2019-01-01 03:15:00     44.612             []           None
14 2019-01-01 03:30:00     44.477       

C:\Users\Lunlo\AppData\Local\Temp\ipykernel_6060\392398549.py:75: RuntimeWarning: Mean of empty slice
  data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))


              DateTime  RealPower RealPower_pred historical_ave
0  2019-01-01 00:00:00     22.362             []           None
1  2019-01-01 00:15:00     22.390             []           None
2  2019-01-01 00:30:00     22.148             []           None
3  2019-01-01 00:45:00     22.022             []           None
4  2019-01-01 01:00:00     22.109             []           None
5  2019-01-01 01:15:00     22.431             []           None
6  2019-01-01 01:30:00     22.125             []           None
7  2019-01-01 01:45:00     22.211             []           None
8  2019-01-01 02:00:00     22.033             []           None
9  2019-01-01 02:15:00     21.982             []           None
10 2019-01-01 02:30:00     22.851             []           None
11 2019-01-01 02:45:00     22.502             []           None
12 2019-01-01 03:00:00     21.976             []           None
13 2019-01-01 03:15:00     22.281             []           None
14 2019-01-01 03:30:00     22.102       

...

C:\Users\Lunlo\AppData\Local\Temp\ipykernel_6060\392398549.py:75: RuntimeWarning: Mean of empty slice
  data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))


              DateTime  RealPower RealPower_pred historical_ave
0  2019-01-01 00:00:00     87.954             []           None
1  2019-01-01 00:15:00     68.636             []           None
2  2019-01-01 00:30:00     68.941             []           None
3  2019-01-01 00:45:00     84.265             []           None
4  2019-01-01 01:00:00     74.297             []           None
5  2019-01-01 01:15:00     68.758             []           None
6  2019-01-01 01:30:00     68.657             []           None
7  2019-01-01 01:45:00     88.111             []           None
8  2019-01-01 02:00:00     68.665             []           None
9  2019-01-01 02:15:00     69.445             []           None
10 2019-01-01 02:30:00     75.354             []           None
11 2019-01-01 02:45:00     81.665             []           None
12 2019-01-01 03:00:00     68.878             []           None
13 2019-01-01 03:15:00     69.034             []           None
14 2019-01-01 03:30:00     85.148       

C:\Users\Lunlo\AppData\Local\Temp\ipykernel_6060\392398549.py:75: RuntimeWarning: Mean of empty slice
  data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))


              DateTime  RealPower RealPower_pred historical_ave
0  2019-01-01 00:00:00    130.539             []           None
1  2019-01-01 00:15:00    125.991             []           None
2  2019-01-01 00:30:00    126.896             []           None
3  2019-01-01 00:45:00    127.176             []           None
4  2019-01-01 01:00:00    127.266             []           None
5  2019-01-01 01:15:00    127.268             []           None
6  2019-01-01 01:30:00    130.487             []           None
7  2019-01-01 01:45:00    128.621             []           None
8  2019-01-01 02:00:00    125.501             []           None
9  2019-01-01 02:15:00    126.377             []           None
10 2019-01-01 02:30:00    127.296             []           None
11 2019-01-01 02:45:00    126.581             []           None
12 2019-01-01 03:00:00    126.879             []           None
13 2019-01-01 03:15:00    131.734             []           None
14 2019-01-01 03:30:00    124.695       

C:\Users\Lunlo\AppData\Local\Temp\ipykernel_6060\392398549.py:75: RuntimeWarning: Mean of empty slice
  data_n['RealPower_pred']=data_n['RealPower_pred'].map(lambda x: np.nanmean(x))


In [ ]:
get_prediction_Simple(
    src_path='../data/UCSD_raw_data/max_intersection_BLD_ALL_temp.csv',
    days_ahead=7,
    start_t='01-Jan-2019',
    end_t='31-Dec-2019',
    save_path = '../data/XGBoost_toy_case/merged_bld_prediction_simple_7days_average.csv',
)

Comparison between Simple prediction and XGBoost prediction

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')

In [ ]:
bld_simple_path='../data/XGBoost_toy_case/merged_bld_prediction_simple_7days_average.csv'
pv_simple_path="../data/XGBoost_toy_case/merged_pv_prediction_simple_7days_average.csv"

bld_xgb_path="../data/XGBoost_toy_case/bld_prediction_XGBoost_07-15.csv"
pv_xgb_path="../data/XGBoost_toy_case/pv_prediction_XGBoost_07-15.csv"

bld_simple=pd.read_csv(bld_simple_path,index_col=0).rename(columns={'RealPower_pred':'Pred_Simple'})
pv_simple=pd.read_csv(pv_simple_path,index_col=0).rename(columns={'RealPower_pred':'Pred_Simple'})
bld_xgb=pd.read_csv(bld_xgb_path,index_col=0).rename(columns={'RealPower_pred':'Pred_XGB'})
pv_xgb=pd.read_csv(pv_xgb_path,index_col=0).rename(columns={'RealPower_pred':'Pred_XGB'})

In [ ]:
bld=pd.merge(bld_simple,bld_xgb,on='DateTime',how='outer')\
    .rename(columns={'RealPower_x':'RealPower'})\
    .drop(columns={"RealPower_y"})
pv=pd.merge(pv_simple,pv_xgb,on='DateTime',how='outer')\
    .rename(columns={'RealPower_x':'RealPower'})\
    .drop(columns={"RealPower_y"})
bld.index = pd.to_datetime(bld.index)
pv.index = pd.to_datetime(pv.index)

In [ ]:
bld

In [ ]:
pv

In [ ]:
def metrics_all(df):
    mse_simple=mean_squared_error(y_true=df['RealPower'],
                                  y_pred=df['Pred_Simple'])
    mse_xgb=mean_squared_error(y_true=df['RealPower'],
                                  y_pred=df['Pred_XGB'])
    mae_simple=mean_absolute_error(y_true=df['RealPower'],
                                  y_pred=df['Pred_Simple'])
    mae_xgb=mean_absolute_error(y_true=df['RealPower'],
                                  y_pred=df['Pred_XGB'])
    def mean_absolute_percentage_error(y_true, y_pred): 
        """Calculates MAPE given y_true and y_pred"""
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mape_simple=mean_absolute_percentage_error(y_true=df['RealPower'],
                                               y_pred=df['Pred_Simple'])
    mape_xgb=mean_absolute_percentage_error(y_true=df['RealPower'],
                                               y_pred=df['Pred_XGB'])
    metric={
        'simple':{
            'mse':mse_simple,
            'mae':mae_simple,
            'mape':mape_simple
        },
        'xgb':{
            'mse':mse_xgb,
            'mae':mae_xgb,
            'mape':mape_xgb
        }
    }
    
    return metric

In [ ]:
metrics_all(bld)

In [ ]:
metrics_all(pv)

In [ ]:
bld.loc['05-01-2019':'05-07-2019']

In [ ]:
def plot_pred(height,width,df,title,lower,upper,linewidth):
    f, ax = plt.subplots(1)
    f.set_figheight(height)
    f.set_figwidth(width)
    #prev_ylim = ax.get_ylim()
    _ = df[['RealPower','Pred_Simple','Pred_XGB']].plot(ax=ax,
                                                style=['-','-','-'],
                                                linewidth=linewidth)
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=10)
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=10)
    ax.set_ylim(lower,upper)
    plot = plt.suptitle(title)
   # ax.set_ylim(prev_ylim) # restore previous ylim
    return plot

def plot_pred_(height,width,df,title,lower,upper,linewidth):
    f, ax = plt.subplots(1)
    f.set_figheight(height)
    f.set_figwidth(width)
    #prev_ylim = ax.get_ylim()
    _ = df[['RealPower','Pred_Simple','Pred_XGB']].plot(ax=ax,
                                                style=['-','-','-'],
                                                linewidth=linewidth)
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=10)
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=10)
    ax.set_ylim(lower,upper)
    plot = plt.suptitle(title)
   # ax.set_ylim(prev_ylim) # restore previous ylim
    return plot

In [ ]:

plot_pred(5,20,bld['10-01-2019':'10-07-2019'],'BLD Load-First Week of Oct',0,500,1.5)

In [ ]:
plot_pred_(5,20,pv['10-01-2019':'10-07-2019'],'PV Load-First Week of Oct',-100,300,1.5)

In [ ]:
df